# ТЕСТИРОВАНИЕ РЕКОМЕНДАТЕЛЬНОЙ СИСТЕМЫ

## 1) Импорт используемых библиотек

Сначала необходимо инпортировать все библиотеки, которые будут использоваться при тестировании.

Для работы с содержимым используемого Google Диска через платформу Google Colab требуется импортировать следующие библиотеки:
- `drive` - модуль, который позволяет подключить Google Диск к виртуальной машине среды выполнения и использовать его содержимое.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Далее импортируем библиотеки, используемые в дальнейшем:
- `pandas` - библиотека для обработки и анализа структурированных данных;
- `numpy` - библиотека, которую применяют для математических вычислений: начиная с базовых функций и заканчивая линейной алгеброй;
- `re` - библиотека, предоставляющая мощные инструменты для работы с текстом.
- `gensim` - библиотека обработки естественного языка предназначения для «Тематического моделирования»;
- `sklearn` - библиотека, реализующая методы машинного обучения, в состав которой входят различные алгоритмы, в том числе предназначенные для задач классификации, регрессионного и кластерного анализа данных, включая метод опорных векторов, метод случайного леса, алгоритм усиления градиента, метод k-средних и DBSCAN;
- `plotly` - графическая библиотека, которая позволяется создавать интерактивные графики.;
- `wordcloud` - библиотека, с помощью которой реализуется метод визуализации данных облако слов, используемый для представления текстовых данных, в котором размер каждого слова указывает на его частоту или важность.

In [3]:
pip install scikit-learn-extra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.5 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd

import re
import collections
import datetime

from gensim.parsing.preprocessing import preprocess_string
from gensim.models.doc2vec import Doc2Vec

from sklearn.manifold import TSNE
from sklearn_extra.cluster import KMedoids
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

import plotly.express as px
from plotly.subplots import make_subplots
from wordcloud import WordCloud

## 2) Подготовка используемых ресурсов

Определим корпус книжных аннотаций и датасет пользовательских предпочтений, которые будут использованы далее при тестировании системы:

In [ ]:
df_user_data = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_user_data.csv',
    index_col='user_id'
)
df_corpus_annotations = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_corpus_annotations.csv',
    index_col='book_id'
)

После определим ранее обученную модель Doc2Vec:

In [ ]:
filename_d2v = '/content/drive/MyDrive/ВКР/children/children_model_d2v.d2v'
model_d2v = Doc2Vec.load(filename_d2v)
VECTOR_SIZE = model_d2v.vector_size

Также определим ранее разработанные класс `UES` и модель `Pref2Vec`:

In [5]:
!python /content/drive/MyDrive/ВКР/user_embeddings_space.py
!python /content/drive/MyDrive/ВКР/pref2vec.py

## 3) Разработка класса Evaluator

Для тестирования разработанного ЯРС так же будет использоваться набор данных веб-сервиса «GoodReads». Проверка работы системы будет проводиться основе датасетов `comics_graphic_user_data.csv` и `children_user_data.csv`. Данные каждой записи о взаимодействиях какого-либо пользователя с различными книгами будут разбиты на две части:
- 80% взаимодействий – выборка X, которая будет использоваться для формирования пользовательского пространства эмбеддингов и на основе которой будет происходить рекомендация книг;
- 20% взаимодействий – выборка Y, которая будет браться за эталон рекомендации.

Для оптимизации процесса проверки работы РС был разработан класс Evaluator, с помощью которого будет производиться подготовка тестовых данных и расчёт значений метрик NCDG@k и Recall@K


In [ ]:
class Evaluator:
    '''
    RECOMMENDER SYSTEM EVALUATOR

    Класс Evaluator позволяет провести тестирование работы рекомендательной
    системы на предмет качества формирования списков рекомендаций. Также этот
    класс позволяет произвести продготовку тестовых данных (данных о
    взаимодействиях пользователей с различными объектами).
    '''


    def __init__(self, model_p2v, test_percent):
        '''
        Метод инициализации класса, в котором производится инициализация полей
        self.model_p2v (модель Pref2Vec) и self.test_percent (процент разбиения
        данных на выборки X и Y).

        Аргументы:
        - model_p2v ('Pref2Vec') - модель Pref2Vec, работа которой будет
            тестироваться;
        - test_percent ('int') - процент разбиения данных на выборки X и Y.
        '''

        self.model_p2v = model_p2v
        self.test_percent = test_percent


    def test(self, users_data, filepath_or_buffer, k=20):
        '''
        Метод, который производит тестирование модели `model_p2v` на
        пользовательских данных `users_data`.

        Используются метрики NDCD@k, Recall@k и Precision@k. Результаты
        тестирования сохраняются в поле `self.df_testing`. Кроме того, имеется
        возможность загрузки результатов в CSV-файл.

        Аргументы:
        - users_data ('pandas.DataFrame') - пользовательские данные,
            используемые при тестировании рекомендательной системы;
        - filepath_or_buffer ('str') - путь к CSV-файлу, в который будут
            сохранены результаты тестирования;
        - k ('int') - параметр 'k', используемый в метриках NDCD@k, Recall@k и
            Precision@k.
        '''

        self.df_testing = pd.DataFrame(columns=['user_id', f"ndcg@{k}", f"recall@{k}", f"precision@{k}"])
        self.k = k

        counter = 1
        for index, row in users_data.iterrows():
            try:
                # Разделение пользовательских данных
                x, y, y_true = self.__sample_preparation(index, row)
                # Получение рекомендаций для текущего пользователя
                recommendations = model_p2v.recommend_for_user(target_user_data=x.iloc[0], topn=k, show_info=False)
                y_pred = [(lambda x: x[0])(x) for x in recommendations]
                # Тестирование и запись результатов в self.df_testing
                ndcg = self.ndcg_at_k(r=[x in y_true for x in y_pred])
                recall = self.recall_at_k(y_true, y_pred)
                precision = self.precision_at_k(y_true, y_pred)
                print(f"{counter}) User {index}:")
                print(f"    ndcg@{k}      = {ndcg}")
                print(f"    recall@{k}    = {recall}")
                print(f"    precision@{k} = {precision}")
                self.df_testing.loc[len(self.df_testing)] = {
                    'user_id': index,
                    f"ndcg@{k}": ndcg,
                    f"recall@{k}": recall,
                    f"precision@{k}": precision
                }
                # Периодическая запись CSV-файла с результатами тестирования
                counter += 1
                if (counter % 5 == 0):
                    self.df_testing.to_csv(filepath_or_buffer)
            except:
                continue

        # Запись CSV-файла с результатами тестирования
        self.df_testing.to_csv(filepath_or_buffer)


    def __sample_preparation(self, user_id, user_data):
        '''
        Метод разбиения тестовых данных на выборки X и Y и ранжирования выборки Y.

        Аргументы:
        - user_id ('str') - идентификатор пользователя, чьи данные будут
            подвержены разбиению;
        - user_data ('pandas.Series') - собственно данные, которые будут
            подвержены разбиению.

        Возвращаются:
        - self.x ('list') - формируемая выборка X;
        - self.y ('list') - формируемая выборка Y;
        - self.y_true ('list') - ранжированная выборка Y;
        '''

        self.x = pd.DataFrame(columns=['user_id', 'read', 'shelved', 'rating_0', 'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5'])
        self.y = pd.DataFrame(columns=['user_id', 'read', 'shelved', 'rating_0', 'rating_1', 'rating_2', 'rating_3', 'rating_4', 'rating_5'])
        row_x = {'user_id': user_id}
        row_y = {'user_id': user_id}

        # Разделение пользовательских данных на выборки X и Y
        for interaction in interactions:
            ids = re.findall('[0-9]+', user_data[interaction])
            x_size = int(len(ids) * (100 - self.test_percent) / 100)
            if (interaction in target_interactions):
                row_x[interaction] = str(ids[:x_size])
                row_y[interaction] = str(ids[x_size:])
            else:
                row_x[interaction] = str(ids)
                row_y[interaction] = str([])
        self.x.loc[0] = row_x
        self.y.loc[0] = row_y

        # Ранжирование выборки Y
        self.y_true = self.__get_y_true(self.x.iloc[0], self.y.iloc[0])

        return (self.x, self.y, self.y_true)


    def __get_y_true(self, x, y):
        '''
        Метод, осуществляющий ранжирование рассматриваемой выборки Y.

        Вычисляется косинусное сходство каждого объекта выборки Y с каждым
        эмбеддингом выборки X, сохранив максимальные значения сходства. Далее
        элементы выборки Y ранжируются по вычисленным сходствам и формируется
        список `y_true`.

        Аргументы:
        - x ('pandas.Series') - выборка X;
        - y ('pandas.Series') - выборка Y.

        Возвращается:
        - self.df_embeggings_y.index.values - ('list') - ранжированный список
            данных выборки Y.
        '''

        # Формирование целевого пользовательского пространства X
        ues_x = UES(x)
        ues_x.df = ues_x.df[ues_x.df['interaction'].isin(target_interactions)]
        self.df_embeggings_x = pd.DataFrame(
            data={'embedding': ues_x.df['embedding'].values},
            index=re.findall('[0-9][0-9]+', str(ues_x.df.index.values))
        )
        # Формирование исследуемого пользовательского пространства Y
        ues_y = UES(y)
        ues_y.df = ues_y.df[ues_y.df['interaction'].isin(target_interactions)]
        self.df_embeggings_y = pd.DataFrame(
            data={'embedding': ues_y.df['embedding'].values, 'distance': [0] * len(ues_y.df['embedding'].values)},
            index=re.findall('[0-9][0-9]+', str(ues_y.df.index.values))
        )
        # Поиск наиболее схожих элементов в пространстве Y с элементами пространства X
        for index, row in self.df_embeggings_y.iterrows():
            self.df_embeggings_y.loc[index, 'distance'] = self.__get_nearest_distance(
                embedding=row['embedding'],
                embedding_id=index
            )
        self.df_embeggings_y = self.df_embeggings_y.sort_values(by='distance', ascending=False).head(self.k)

        return self.df_embeggings_y.index.values


    def __get_nearest_distance(self, embedding, embedding_id):
        '''
        Метод, при помощи которого вычисляется косинусное сходство эмбеддинга
        объекта выборки Y с каждым эмбеддингом выборки X. Выводит максимальное
        значение сходства.

        Аргументы:
        - embedding ('list') - эмбеддинг рассматриваемого объекта;
        - embedding_id ('str') - индентификатор рассматриваемого объекта.

        Возвращается:
        - self.df_embeggings_x['distance'].max() ('float') - максимальное
            значение сходства.
        '''

        # Рассчёт растояний между рассматриваемым объектом и целевыми
        distances = cosine_similarity([embedding], list(self.df_embeggings_x['embedding']))
        self.df_embeggings_x['distance'] = np.reshape(distances, self.df_embeggings_x['embedding'].size)

        # Обработка случая, в котором найденное рассторие - расстояние с искомым объектом
        if (self.df_embeggings_x['distance'].idxmax() == embedding_id):
            self.df_embeggings_x.drop(self.df_embeggings_x['distance'].idxmax())

        return self.df_embeggings_x['distance'].max()


    def precision_at_k(self, y_true, y_pred):
        '''
        Метод, позволяющий получить значение метрики Precision@k.

        Аргументы:
        - y_true ('list') - эталонный список рекомендаций;
        - y_pred ('list') - список рекомендаций, формируемый
            рекомендательной системой;

        Возвращает:
        - precision ('float') - значение метрики Precision@k
        '''

        set_y_true = set(y_true)
        set_y_pred = set(y_pred[:self.k])

        precision = len(set_y_true & set_y_pred) / float(self.k)

        return precision


    def recall_at_k(self, y_true, y_pred):
        '''
        Метод, позволяющий получить значение метрики Recall@k.

        Аргументы:
        - y_true ('list') - эталонный список рекомендаций;
        - y_pred ('list') - список рекомендаций, формируемый
            рекомендательной системой;

        Возвращает:
        - recall ('float') - значение метрики Recall@k
        '''

        set_y_true = set(y_true)
        set_y_pred = set(y_pred[:self.k])

        recall = len(set_y_true & set_y_pred) / float(len(set_y_true))

        return recall


    def dcg_at_k(self, r, method=0):
        '''
        Метод, позволяющий получить значение метрики DCG@k.

        Аргументы:
        - y_true ('list') - эталонный список рекомендаций;
        - y_pred ('list') - список рекомендаций, формируемый
            рекомендательной системой;

        Возвращает:
        - dcg ('float') - значение метрики DCG@k
        '''

        r = np.asfarray(r)[:self.k]

        if r.size:
            if method == 0:
                return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
            elif method == 1:
                return np.sum(r / np.log2(np.arange(2, r.size + 2)))
            else:
                raise ValueError('method must be 0 or 1.')

        return 0.


    def ndcg_at_k(self, r, method=0):
        '''
        Метод, позволяющий получить значение метрики NDCG@k.

        Аргументы:
        - y_true ('list') - эталонный список рекомендаций;
        - y_pred ('list') - список рекомендаций, формируемый
            рекомендательной системой;

        Возвращает:
        - ndcg ('float') - значение метрики NDCG@k
        '''

        dcg_max = self.dcg_at_k(sorted(r, reverse=True), method)

        if not dcg_max:
            return 0.

        return self.dcg_at_k(r, method) / dcg_max


Процесс тестирования модели Pref2Vec при помощи класса Evaluator представляет из себя следующую последовательность действий:
1.	Пользовательские данные для одного пользователя разбиваются на выборки X, Y;
2.	Выборка Y ранжируется относительно выборки X;
3.	Выборка X подаётся на вход модели Pref2Vec, вычисляются рекомендации;
4.	Полученный список рекомендаций сравнивается с ранжированной выборкой Y посредством метрик NCDG@k и Recall@k;
5.	Вышеописанный алгоритм повторяется для N пользователей. После вычисляются средние значения метрик NCDG@k и Recall@k и выводятся как результат тестирования.

Далее приведём пример работы класса. Для тестирования будут использоваться данные пользователей, чья последовательность действий относительно длинная:

In [ ]:
df_user_data[df_user_data['rating_5'].str.len() > 400].iloc[:2]

,read,shelved,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5
user_id,,,,,,,,
bafc2d50014200cda7cb2b6acd60cd73,[31301849 30272307 137894 31672148 31258181 ...,[18430804 18738869 15704307 6954438 389519 ...,[11476305 17137616 9041662 1098336],[],[],[18430203 59966 13094398 13526176 163377 ...,[ 137894 31672148 31258181 30810333 30287874 ...,[31301849 30272307 26778322 13084667 25266691 ...
a5ac0c0a728259db23e16e8e143e3325,[26247019 25667060 24464154 24464123 451932 ...,[ 954242 954243 954244 954240 954245 ...,[26247019 25667060 24464154 24464123],[],[],[8315979 6664559 717201 1382251 500463 2824...,[12111331 26050510 9696114 6296507 9696113 ...,[ 451932 1949778 1311025 1311028 1311026 ...


In [ ]:
df_user_data[df_user_data['rating_5'].str.len() > 400].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2998 entries, 8842281e1d1347389f2ab93d60773d4d to 4912f4527840a57bd2ce58abb17a24c1
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   read      2998 non-null   object
 1   shelved   2998 non-null   object
 2   rating_0  2998 non-null   object
 3   rating_1  2998 non-null   object
 4   rating_2  2998 non-null   object
 5   rating_3  2998 non-null   object
 6   rating_4  2998 non-null   object
 7   rating_5  2998 non-null   object
dtypes: object(8)
memory usage: 210.8+ KB


Пример запуска тестирования:

In [ ]:
model_p2v = Pref2Vec(depth=500)
model_p2v.load_corpus(filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_corpus_pref2vec.csv')

evaluator = Evaluator(model_p2v=model_p2v, test_percent=20)
evaluator.test(
    users_data=df_user_data[df_user_data['rating_5'].str.len() > 400].iloc[:200],
    k=10,
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_testing_depth_500_percent_20_k_10_1.csv'
)

## 4) Собственно тестирование

Было произведено тестирование модели Pref2Vec на наборах данных «Children» и «Comics & Graphic». Приведём результаты тестирования на данных «Children»:

In [ ]:
df_children_test = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/children_test.csv',
    index_col='user_id'
)

In [ ]:
dict_recall = {
    'name':  ['BPRMF', 'BPRMF', 'BPRMF',
              'GRU4Rec', 'GRU4Rec', 'GRU4Rec',
              'GRU4Rec+', 'GRU4Rec+', 'GRU4Rec+',
              'NextItNet', 'NextItNet', 'NextItNet',
              'Caser', 'Caser', 'Caser',
              'SASRec', 'SASRec', 'SASRec',
              'HGN', 'HGN', 'HGN',
              'Pref2Vec', 'Pref2Vec', 'Pref2Vec'],
    'k':     [10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20],
    'value': [0.082, 0.108, 0.131,
              0.086, 0.113, 0.137,
              0.095, 0.122, 0.142,
              0.088, 0.115, 0.140,
              0.105, 0.141, 0.164,
              0.120, 0.159, 0.172,
              0.133, 0.170, 0.183,
              df_children_test['recall@10'].mean(),
              df_children_test['recall@15'].mean(),
              df_children_test['recall@20'].mean()]
}
df_recall = pd.DataFrame(dict_recall)

fig = px.bar(df_recall, x="k", y="value", color='name', barmode='group',
             labels={'value': 'Recall@k', 'name': 'Модели:'})
fig.show()

In [ ]:
dict_ndcg = {
    'name':  ['BPRMF', 'BPRMF', 'BPRMF',
              'GRU4Rec', 'GRU4Rec', 'GRU4Rec',
              'GRU4Rec+', 'GRU4Rec+', 'GRU4Rec+',
              'NextItNet', 'NextItNet', 'NextItNet',
              'Caser', 'Caser', 'Caser',
              'SASRec', 'SASRec', 'SASRec',
              'HGN', 'HGN', 'HGN',
              'Pref2Vec', 'Pref2Vec', 'Pref2Vec'],
    'k':     [10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20],
    'value': [0.065, 0.058, 0.042,
              0.072, 0.064, 0.050,
              0.084, 0.078, 0.061,
              0.073, 0.063, 0.052,
              0.096, 0.084, 0.078,
              0.102, 0.090, 0.085,
              0.115, 0.103, 0.096,
              df_children_test['ndcg@10'].mean(),
              df_children_test['ndcg@15'].mean(),
              df_children_test['ndcg@20'].mean()]
}
df_ndcg = pd.DataFrame(dict_ndcg)

fig = px.bar(df_ndcg, x="k", y="value", color='name', barmode='group',
             labels={'value': 'NDCG@k', 'name': 'Модели:'})
fig.show()

После приведём результаты тестирования на данных «Comics & Graphic»:

In [ ]:
df_comics_graphic_test = pd.read_csv(
    filepath_or_buffer='/content/drive/MyDrive/ВКР/children/comics_graphic_test.csv',
    index_col='user_id'
)

In [ ]:
dict_recall = {
    'name':  ['BPRMF', 'BPRMF', 'BPRMF',
              'GRU4Rec', 'GRU4Rec', 'GRU4Rec',
              'GRU4Rec+', 'GRU4Rec+', 'GRU4Rec+',
              'NextItNet', 'NextItNet', 'NextItNet',
              'Caser', 'Caser', 'Caser',
              'SASRec', 'SASRec', 'SASRec',
              'HGN', 'HGN', 'HGN',
              'Pref2Vec', 'Pref2Vec', 'Pref2Vec'],
    'k':     [10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20],
    'value': [0.072, 0.1,   0.12,
              0.095,  0.13, 0.149,
              0.13,  0.155, 0.164,
              0.11,  0.143, 0.157,
              0.148, 0.167, 0.191,
              0.15,  0.172, 0.205,
              0.172, 0.205, 0.216,
              df_comics_graphic_test['recall@10'].mean(),
              df_comics_graphic_test['recall@15'].mean(),
              df_comics_graphic_test['recall@20'].mean()]
}
df_recall = pd.DataFrame(dict_recall)

fig = px.bar(df_recall, x="k", y="value", color='name', barmode='group',
             labels={'value': 'Recall@k', 'name': 'Модели:'})
fig.show()

In [ ]:
dict_ndcg = {
    'name':  ['BPRMF', 'BPRMF', 'BPRMF',
              'GRU4Rec', 'GRU4Rec', 'GRU4Rec',
              'GRU4Rec+', 'GRU4Rec+', 'GRU4Rec+',
              'NextItNet', 'NextItNet', 'NextItNet',
              'Caser', 'Caser', 'Caser',
              'SASRec', 'SASRec', 'SASRec',
              'HGN', 'HGN', 'HGN',
              'Pref2Vec', 'Pref2Vec', 'Pref2Vec'],
    'k':     [10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20,
              10, 15, 20],
    'value': [0.065, 0.06,  0.055,
              0.084, 0.064, 0.058,
              0.125, 0.107, 0.08,
              0.116, 0.07,  0.061,
              0.157, 0.14,  0.12,
              0.150, 0.136, 0.119,
              0.194, 0.16,  0.145,
              df_comics_graphic_test['ndcg@10'].mean(),
              df_comics_graphic_test['ndcg@15'].mean(),
              df_comics_graphic_test['ndcg@20'].mean()]
}
df_ndcg = pd.DataFrame(dict_ndcg)

fig = px.bar(df_ndcg, x="k", y="value", color='name', barmode='group',
             labels={'value': 'NDCG@k', 'name': 'Модели:'})
fig.show()